**Configure Devices**

In [1]:
import sys
import tensorflow as tf
sys.path.append('./src/')
sys.path.append('./rcwa_tf/src/')
import utils

In [2]:
# Limit GPU memory growth.
tf_utils.config_gpu_memory_usage()

# Choose the device to run on.
use_gpu = True
##### REMEMBER: CHANGE BACK TO GPU:1
tfDevice = '/job:localhost/replica:0/task:0/device:GPU:1' if use_gpu else '/CPU:0'
print('Executing on device ' + tfDevice + ' ...')

# Measure GPU memory usage.
gpu_memory_init = tf_utils.gpu_memory_info()

 
2 Physical GPUs
Executing on device /job:localhost/replica:0/task:0/device:GPU:1 ...


**Dependencies**

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import solver
import solver_metasurface
import time

**Loss Function Definition**

In [4]:
def loss_function(h, params):
    
    # Generate permittivity and permeability distributions.
    ER_t, UR_t = solver_metasurface.generate_layered_metasurface(h, params)

    # Simulate the system.
    outputs = solver.simulate(ER_t, UR_t, params)
    
    # First loss term: maximize sum of electric field magnitude within some radius of the desired focal point.
    r = params['focal_spot_radius']
    field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
    focal_plane = solver.propagate(params['input'] * field, params['propagator'], params['upsample'])
    index = (params['pixelsX'] * params['upsample']) // 2
    l1 = tf.math.reduce_sum(tf.abs(focal_plane[0, index-r:index+r, index-r:index+r]))

    # Final loss: (negative) field intensity at focal point + field intensity elsewhere.
    return -params['w_l1']*l1

**Initialize User-Configurable Parameters**

In [5]:
with tf.device(tfDevice):
    
    # Initialize dict of user-configurable parameters.
    user_params = {}
    
    # Flage to enable hyperparameter grid search.
    user_params['enable_hyperparameter_gridsearch'] = True
    
    # Tunable parameters.
    user_params['N'] = 10
    user_params['sigmoid_update'] = 20.0
    user_params['learning_rate'] = 4E-1
    user_params['initial_height'] = 0
    
    user_params['parameter_string'] = 'N' + str(user_params['N']) \
        + '-sigmoid_update' + str(user_params['sigmoid_update']) \
        + '-learning_rate' + str(user_params['learning_rate']) \
        + '-initial_height' + str(user_params['initial_height'])
    
    # Values to use in hyperparameter grid search.
    # Stored as a dict. Each dict key is the key in user_params corresponding to a tunable
    # hyperparameter, i.e. 'N' and its value is a list of values to try for that hyperparameter.
    param_grid = {'N': [10],
                  'sigmoid_update': [10.0, 20.0],
                  'learning_rate': [2E-1],
                  'initial_height': [0]}
    user_params['param_grid'] = param_grid
    
    
    # Source parameters.
    user_params['wavelengths'] = [120000.0]
    user_params['thetas'] = [0.0]
    user_params['phis'] = [0.0]
    user_params['pte'] = [1.0]
    user_params['ptm'] = [0.0]
    
    # Device parmeters.
    user_params['pixelsX'] = 18
    user_params['pixelsY'] = user_params['pixelsX']
    user_params['erd'] = 12.04
    user_params['ers'] = user_params['erd']
    user_params['L'] = [50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 950000.0]
    user_params['Lx'] = 20000.0
    #user_params['Lx'] = 5000000. / user_params['pixelsX']
    user_params['Ly'] = user_params['Lx']
    user_params['f'] = 0.0 # Focal distance (nm)
    
    # Solver parameters.
    user_params['PQ'] = [5,5]
    user_params['upsample'] = 11
    
    # Problem parameters.
    user_params['w_l1'] = 1.0
    user_params['sigmoid_coeff'] = 1.0
    user_params['focal_spot_radius'] = 10
    user_params['enable_random_init'] = False
    user_params['enable_debug'] = False
    user_params['enable_print'] = True
    user_params['enable_timing'] = True
    
    # Loss function.
    user_params['loss_function'] = loss_function
    
    # Logging parameters.
    user_params['enable_logging'] = True
    user_params['log_filename_prefix'] = 'nearfield-' + str(user_params['pixelsX']) + 'x' + str(user_params['pixelsY']) + '-'
    user_params['log_filename_extension'] = '.log'
    user_params['load_result'] = False
    user_params['load_filename'] = user_params['log_filename_prefix'] + user_params['parameter_string'] + user_params['log_filename_extension']

**Run Tests**

In [6]:
with tf.device(tfDevice):
    
    if user_params['enable_timing']:
        start_time = time.time()
            
    if user_params['enable_hyperparameter_gridsearch']:
        
        # Perform the hyperparameter grid search.
        results = solver_metasurface.hyperparameter_gridsearch(user_params)
        
        # Get list of evaluation scores.
        scores = [r['eval_score'] for r in results]

        # Select hyperparameters corresponding to best evaluation score.
        result = results[np.argmax(scores)]
        h = result['h']
        loss = result['loss']
        focal_plane = result['focal_plane']
        eval_score = result['eval_score']
        params = result['params']

        if user_params['enable_print']:
            print('Best hyperparameters: ' + str(result['hyperparameters']))
            print('With evaluation score: ' + str(eval_score))
    
    elif user_params['load_result']:
        
        if user_params['enable_print']:
            print('Loading results from file ' + user_params['load_filename'] + ' .')
        
        result = solver_metasurface.load_result(user_params['load_filename'])
        h = result['h']
        loss = result['loss']
        focal_plane = result['focal_plane']
        eval_score = result['eval_score']
        
    else:
        
        # Optimize.
        h, loss, params = solver_metasurface.optimize_device(user_params)
        
        # Get full device representation of resulting metasurface.
        ER_t, UR_t = solver_metasurface.generate_layered_metasurface(h, params)
        
        # Simulate scattering from the optimized device.
        outputs = solver.simulate(ER_t, UR_t, params)
        field = outputs['ty'][:, :, :, np.prod(params['PQ']) // 2, 0]
        focal_plane = solver.propagate(params['input'] * field, params['propagator'], params['upsample'])
        eval_score = solver_metasurface.evaluate_solution(focal_plane, params)
        
        # Compile results and output to a log file.
        result = {'hyperparameter_names': ['N', 'sigmoid_update', 'learning_rate', 'initial_height'],
            'hyperparameters': [user_params['N'], user_params['sigmoid_update'], user_params['learning_rate'], user_params['initial_height']],
            'h': h,
            'loss': loss,
            'focal_plane': focal_plane,
            'eval_score': eval_score}
        solver_metasurface.log_result(result, user_params['log_filename_prefix'] + user_params['parameter_string'] + user_params['log_filename_extension'])
    
    if user_params['enable_timing']:
        print('Completed tests in ' + str(time.time() - start_time) + ' s.')

Beginning hyperparameter grid search...

Trying hyperparameters: ['N', 'sigmoid_update', 'learning_rate', 'initial_height']
(10, 10.0, 0.2, 0)
Optimizing... Iteration 0, 

ResourceExhaustedError: OOM when allocating tensor with shape[1,50,50,6] and type complex64 on /job:localhost/replica:0/task:0/device:GPU:1 by allocator GPU_1_bfc [Op:StridedSlice] name: strided_slice/

**Display Learning Curve**

In [ ]:
with tf.device(tfDevice):
    
    plt.plot(loss)
    plt.xlabel('Iterations')
    plt.ylabel('Loss')
    plt.show()

**Display Resulting Intensity on the Focal Plane**

In [ ]:
with tf.device(tfDevice):
    
    plt.imshow(tf.abs(focal_plane[0, :, :]) ** 2)
    plt.colorbar()

In [ ]:
with tf.device(tfDevice):
    
    index = (user_params['pixelsX'] * user_params['upsample']) // 2
    plt.plot(tf.abs(focal_plane[0, index, :]) ** 2)
    plt.xlabel('x')
    plt.ylabel('Electric Field Intensity on Focal Plane')
    plt.show()

In [ ]:
with tf.device(tfDevice):
    
    index = (user_params['pixelsX'] * user_params['upsample']) // 2
    plt.plot(tf.abs(focal_plane[0, :, index]) ** 2)
    plt.xlabel('y')
    plt.ylabel('Electric Field Intensity on Focal Plane')
    plt.show()

**Display Metasurface**

In [ ]:
with tf.device(tfDevice):
    
    ER_t, UR_t = solver_metasurface.generate_layered_metasurface(h, params)
    solver_metasurface.display_layered_metasurface(ER_t, params)
    print(h)

**Check GPU Memory Utilization**

In [ ]:
gpu_memory_final = tf_utils.gpu_memory_info()
gpu_memory_used = [gpu_memory_final[1][0] - gpu_memory_init[1][0], gpu_memory_final[1][1] - gpu_memory_init[1][1]]
print('Memory used on each GPU(MiB): ' + str(gpu_memory_used))